In [ ]:
## Colab
!pip install langchain openai

import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/work_space/Paper/NLP_2023')
with open('./Openai_API_Key.txt', 'r') as api:
    os.environ["OPENAI_API_KEY"] = api.read()

In [4]:
## Local
import os
with open('./Openai_API_Key.txt', 'r') as api:
    os.environ["OPENAI_API_KEY"] = api.read()

In [5]:
from util.module import *

In [7]:
positive_chat('완전 재밌어요 ㅋㅋ .. 이거 또 보고 싶어요', verbose=True)



> Entering new LLMChain chain...
Prompt after formatting:
System: 당신은 긍정적인 리뷰에 답변을 달아주는 유용한 AI 봇입니다. 상대방의 칭찬에 감사와 고마움을 표하는 답변을 작성하세요.
Human: 믿고쓰는 상품! 너무나도 만족합니다. 항상 좋은 제품 제공해주셔서 너무나도 감사합니다. 배송도 빠르고 서비스도 너무 좋아요 ^^
AI: 안녕하세요 고객님, 항상 저희 제품을 사용해주셔서 대단히 감사합니다. 앞으로도 좋은 서비스로 보답할 수 있도록 하겠습니다. 고맙습니다.
Human: 완전 재밌어요 ㅋㅋ .. 이거 또 보고 싶어요

> Finished chain.


'안녕하세요, 좋아해주셔서 감사합니다! 당신의 행복이 우리의 행복입니다. 앞으로도 재미있는 콘텐츠를 제공하도록 노력하겠습니다. 다시 한번 이용해 주셔서 감사드립니다!'

In [4]:
from util.chain_model import sentiment_chain_model

In [5]:
sentiment_chain_model('빠른 배송과 친절한 설명', show_sentiment=True, show_emotion=True)

{'Sentiment': 'Positive',
 'Emotion': 'Happiness',
 'Answer': '고객님의 칭찬에 감사드립니다! 저희는 고객님의 만족을 위해 항상 노력하고 있습니다. 앞으로도 꾸준한 관심과 사랑 부탁드리며, 항상 최선을 다하겠습니다. 감사합니다!'}

In [4]:
sentiment_chain_model('빠른 배송과 친절한 설명', show_sentiment=False)

'고객님의 긍정적인 리뷰에 감사드립니다! 더욱 빠른 배송과 친절한 설명으로 고객님께 만족감을 드릴 수 있도록 노력하겠습니다. 항상 저희 서비스를 이용해주셔서 감사합니다.'

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain

def sentiment_chat(text:str):
  senti_chat = ChatOpenAI(model_name='gpt-4', temperature = 0)
  schema = {
      "properties": {
          "sentiment" : {"type" : "string", "enum" : ['Very Positive', 'Positive', 'Neutral', 'Negative', 'Verey Negative']},
          "aggressiveness": {"type" : 'integer', "enum" : [1,2,3,4,5], "description" : "describes how aggressive the statement is, the higher the more aggressive"}
      },
      "required" : ["sentiment", "aggressiveness"]
  }
  senti_chain = create_tagging_chain(schema, senti_chat)
  answer = senti_chain.run(text)

  return answer

def emotion_chat(text:str):
  emo_chat = ChatOpenAI(model_name='gpt-4', temperature = 0)
  schema = {
      "properties" : {
          "emotion" : {"type" : "string", "enum" : ['Anger', 'Disgust', 'Fear', 'Happiness', 'Contempt', 'Sadness', "Surprise"]},
          "aggressiveness": {"type" : 'integer', "enum" : [1,2,3,4,5], "description" : "describes how aggressive the statement is, the higher the more aggressive"}
      }
  }
  emo_chain = create_tagging_chain(schema, emo_chat)
  answer = emo_chain.run(text)

  return answer

def positive_chat(text:str, emotion:bool=False, verbose:bool=False):
    if emotion:
        user_emotion = emotion_chat(text)['emotion']
        positive_memory = ConversationBufferWindowMemory(k=1, memory_key="chat_history", return_messages=True)
        positive_memory.save_context({"input" : "믿고쓰는 상품! 너무나도 만족합니다. 항상 좋은 제품 제공해주셔서 너무나도 감사합니다. 배송도 빠르고 서비스도 너무 좋아요 ^^"},
        {"output" : "안녕하세요 고객님, 항상 저희 제품을 사용해주셔서 대단히 감사합니다. 앞으로도 좋은 서비스로 보답할 수 있도록 하겠습니다. 고맙습니다."})

        llm = ChatOpenAI(model_name='gpt-4', temperature = 0.8)
        prompt = ChatPromptTemplate(
            input_variables = ['user_emotion', 'review', 'chat_history'],
            messages = [
                SystemMessagePromptTemplate.from_template("당신은 긍정적인 리뷰에 답변을 달아주는 유용한 AI 봇입니다. 상대방의 칭찬에 감사와 고마움을 표하는 답변을 작성하세요."),
                MessagesPlaceholder(variable_name="chat_history"),
                HumanMessagePromptTemplate.from_template("사용자의 감정 = {user_emotion}, 사용자의 리뷰 = {review}")
                ]
            )

        model = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=verbose,
        memory=positive_memory
        )

        answer = model.run(user_emotion=user_emotion, review= text)['text']
        return answer

In [26]:
positive_chat('완전 재밌어요 ㅋㅋ .. 이거 또 보고 싶어요', emotion=True, verbose=True)



> Entering new LLMChain chain...
Prompt after formatting:
System: 당신은 긍정적인 리뷰에 답변을 달아주는 유용한 AI 봇입니다. 상대방의 칭찬에 감사와 고마움을 표하는 답변을 작성하세요.
Human: 믿고쓰는 상품! 너무나도 만족합니다. 항상 좋은 제품 제공해주셔서 너무나도 감사합니다. 배송도 빠르고 서비스도 너무 좋아요 ^^
AI: 안녕하세요 고객님, 항상 저희 제품을 사용해주셔서 대단히 감사합니다. 앞으로도 좋은 서비스로 보답할 수 있도록 하겠습니다. 고맙습니다.
Human: 사용자의 감정 = Happiness, 사용자의 리뷰 = 완전 재밌어요 ㅋㅋ .. 이거 또 보고 싶어요

> Finished chain.


ValueError: One input key expected got ['review', 'user_emotion']